## RSNA-MICCAI Brain Tumor Radiogenomic Classification
<a id = 0></a>

A malignant tumor in the brain is a life-threatening condition. Known as glioblastoma, it's both the most common form of brain cancer in adults and the one with the worst prognosis, with median survival being less than a year. The presence of a specific genetic sequence in the tumor known as MGMT promoter methylation has been shown to be a favorable prognostic factor and a strong predictor of responsiveness to chemotherapy

![](https://images.pexels.com/photos/4226119/pexels-photo-4226119.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)

#           [Contents](#1)
* [Title](#0)
* [Dependencies](#1) 
* [Introduction](#2)
* [MGMT Label](#3)
* [Visualizing the MRI](#4)
* [Animations](#5)


# Dependencies
<a id = 1></a>

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pydicom # A library that loads dicom(dcm) files
import numpy as np
import os
import glob
from IPython.display import Markdown # we will require this to print Markdown in the console
input_df = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv"
sample = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv"


In [ ]:
!pip install celluloid #The library celluloid is not available in the default kaggle enviroment so we have to pip install it

Let's start by checking the train csv and sample csv

In [ ]:
train_df = pd.read_csv(input_df)
train_df

In [ ]:
sample_df = pd.read_csv(sample)
sample_df

# Introduction 
<a id = 2></a>

 Having a First Glance at the two datasets. I can summarize that each row BraTS21ID corresponds to a patient and the MGMT_value column is the probability that the patient has the methylation promoter which is a cancer causing gene. 1 means a 100% probability that a person has the gene and 0 means that the probability that the person has the gene is 0 % . There is a test file and train file which have the coressponding BraTS21ID files each containing images in the Dicom format.
 
 
 
Our job is to find the relations between the images in the train set and the likelihood that the person has the methylation promoter gene. Then use the same relation to predict using the images in the test set the probability of that paitent having the methylation of the MGMT promoter gene

# MGMT Label 
<a id = 3></a>

In [ ]:
sns.countplot(data = train_df , x = "MGMT_value")

The values of MGMT_value which are 1 and 0 are close. Which is great while making the model the model will not tend to make a predicton which has a greater likelihood of occuring.

# Visualizing the MRI
<a id = 4></a>

In [ ]:
def load_dicom(path):
    dicom = pydicom.read_file(path) # pydicom is a library made to process dicom files
    data = dicom.pixel_array # pixel_array shows the value of each pixel in the array
    data = data - np.min(data) #helps in normalizing the image (not really required in this case)
    if np.max(data) != 0:
        data = data/np.max(data)
    data = (data*255).astype(np.uint8)
    return data

def visualize_sample(ID,path,type_ = "flair"):
    plt.figure(figsize = (16,5))
    data = load_dicom(path)
    plt.imshow(data)#
    label = train_df[train_df['BraTS21ID'] == ID]["MGMT_value"].item()#this will assign the MGMT_value of the Patient to label
    plt.title(str(ID)+" " + type_ + " MGMT_value: " +str(label))
    plt.axis("off")

We'll first check the MRI's of patient 0000 who as we can see on the graph has the MGMT promoter methalation

In [ ]:
visualize_sample(0 , "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/FLAIR/Image-250.dcm", "FLAIR")

FLAIR MRI :- **Fl**uid **A**ttenuated **I**nversion **r**ecovery . It is a technique that shows tissue as white and cerebrospinal fluid as darkened parts of the brain

In [ ]:
display(visualize_sample(0 , "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1w/Image-15.dcm", "T1w"))
display(visualize_sample(0 , "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T2w/Image-250.dcm", "T2w"))

T1w and T2w:- From my understanding T1w and t2w shows tissue matter of the brain. The difference between them is than T1w has a lower relaxation time (The model of the brain is made using fewer images)

In [ ]:
visualize_sample(0 , "../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/00000/T1wCE/Image-75.dcm", "T1wCE")

These Images are great they were however made to be mapped as a 3d model of the brain. Now lets animate these images in the next section

# Animations
<a id = 5></a>

In [ ]:
from celluloid import Camera
from IPython.display import HTML
from matplotlib import animation, rc
rc('animation', html='jshtml')


def create_animation(ims):
    fig = plt.figure(figsize=(16, 5))
    plt.axis('off')
    im = plt.imshow(ims[0])
    camera = Camera(fig)

    for i in range(len(ims)):
        plt.imshow(ims[i])
        camera.snap() #Each instance of the plot is saved
    
    animation = camera.animate() # And is then animated
    display(animation)

#Although I find celluloid much easier to use matplotlib's animation function Is much more intuitive to use
# def create_animation(ims):
#     fig = plt.figure(figsize=(6, 6))
#     plt.axis('off')
#     im = plt.imshow(ims[0])

#     def animate_func(i):
#         im.set_array(ims[i])
#         return [im]

#     return display(animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//24))

In [ ]:
def dicom_animation(path):
    t_path = sorted(
                    glob.glob(os.path.join(path,'*'))
    )
    images = []
    for name in t_path:
        data = load_dicom(name)
        images.append(data)
    return create_animation(images)

In [ ]:
types = ["FLAIR", "T1w" , "T1wCE" , "T2w"]
def show_anim(Id):
    display(Markdown("<div align = 'center'  style = 'color : lightblue'>"+"ID : "+Id+"</div>"))
    display(Markdown("<div align = 'center'  style = 'color :lightblue'>"+"MGMT methylation promoter : "+str(train_df[train_df['BraTS21ID'] == int(Id)]["MGMT_value"].item())+"</div>"))
    #print("\tMGMT methylation promoter : " ,train_df[train_df['BraTS21ID'] == int(Id)]["MGMT_value"].item())
    path = os.path.join("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train",Id)
    for x in types:
        display(Markdown("<div align = 'center' style = 'color : violet'>"+"Type of MRI : "+x+"</div>"))
        dicom_animation(os.path.join(path,x))
    

In [ ]:
show_anim("00000")

Now let's check the same with a patient who has the a MRNA promoter methalation 

In [ ]:
show_anim("00003")